In [4]:
import os,sys,string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Success


# Loading in the equivalent Majorana uncertainties

In [5]:
Params = {"Run":"run1", #Only have Dirac samples for Run1
          "Load_lepton_dirac":False,
          "Load_pi0_dirac":True} 

Run = "run1"

loc_hists = "Uncertainties/"

if Params["Load_lepton_dirac"] == True: 
    HNL_masses = Constants.HNL_ee_dirac_mass_samples
    name_type = "ee"
if Params["Load_pi0_dirac"] == True: 
    HNL_masses = Constants.HNL_pi0_dirac_mass_samples
    name_type = "pi0"
    loc_hists += "pi0/"

BDT_name = "_full_Finished_10"
filename = BDT_name
print(filename)

uncertainties_dict = {}

root_filename = name_type+filename
bins_dict, bin_cents = {}, {}
for HNL_mass in HNL_masses:
    hist_placeholder = uproot.open(loc_hists+f'{Run}_{HNL_mass}_'+root_filename+".root")
    bins_dict[HNL_mass] = hist_placeholder['bkg_overlay'].to_numpy()[1] #A tuple of bin edges
    bin_cents[HNL_mass]=(bins_dict[HNL_mass][:-1]+bins_dict[HNL_mass][1:])/2
    
    uncertainties_placeholder = {'ppfx_uncertainty_frac': hist_placeholder['ppfx_uncertainty_frac'],
                                 'Genie_uncertainty_frac': hist_placeholder['Genie_uncertainty_frac'],
                                 'Reinteraction_uncertainty_frac': hist_placeholder['Reinteraction_uncertainty_frac'],
                                 "overlay_DetVar_uncertainty_frac": hist_placeholder["overlay_DetVar_uncertainty_frac"],
                                 "signal_DetVar_uncertainty_frac": hist_placeholder["signal_DetVar_uncertainty_frac"]}
    
    uncertainties_dict[HNL_mass] = uncertainties_placeholder
    

_full_Finished_10


## Writing the Majorana uncertainties to the Dirac files

In [6]:
# run1_245_pi0_dirac_full_Finished_10.root
Dirac_name_type = name_type + "_dirac"
Dirac_filename = Dirac_name_type+filename
for HNL_mass in HNL_masses:
    
    hist_samples = uncertainties_dict[HNL_mass]
    
    stop_writing = False
    dont_save = []
    with uproot.open(loc_hists+Run+f'_{HNL_mass}_{Dirac_filename}.root') as file: #Check what is already in the file (read-only)
        for name in hist_samples:
            if str(name)+";1" in file.keys():
                # print(str(name) + f" ALREADY EXISTS in {HNL_mass}MeV file, not re-saving")
                dont_save.append(name)

    with uproot.update(loc_hists+f'{Run}_{HNL_mass}_{Dirac_filename}.root') as file: #Add new hists into the file
        # del file['ppfx_uncertainty']
        for name in hist_samples:
            # if stop_writing == True:
            #     print("Not saving hist")
            #     break
            if name in dont_save:
                # print(f"Not saving {name}")
                continue
            else:
                file[name] = hist_samples[name]
                
print("Done")

Done


In [ ]:
for HNL_mass in HNL_masses:
    with uproot.open(loc_hists+Run+f'_{HNL_mass}_{Dirac_filename}.root') as file:
    # with uproot.open(loc_hists+f'{Run}_{HNL_mass}MeV_{filename}.root') as file: #Check what is already in the file (read-only)
        print(file.keys())
        print(file['signal_DetVar_uncertainty_frac;1'].values())
        

In [ ]:
def make_overflow_bin(bins_dict, bins_cents_dict):
    """
    For making the final "overflow" bin the same size as the previous bins, i.e one integer in width.
    """
    bins_overflow, bins_cent_overflow = {}, {}
    for HNL_mass in bins_dict:
        overflow_bin = bins_cents_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin centre val. 
        bins_cent_overflow[HNL_mass] = bins_cents_dict[HNL_mass].copy()
        bins_cent_overflow[HNL_mass][-1] = overflow_bin
        bins_overflow[HNL_mass] = bins_dict[HNL_mass].copy()
        bins_overflow[HNL_mass][-1] = bins_dict[HNL_mass][-2]+1 #Just adding one to the penultimate bin end val. 
    return bins_overflow, bins_cent_overflow

bins_overflow, bins_cents_overflow = make_overflow_bin(bins_dict, bin_cents)

def make_xlims_dict(bins_dict, spacing, lower = None):
    """
    Making a dict of xlims for plotting several mass points at once.
    Also returns a dict of xticks for the purpose of indicating the overflow.
    """
    xlims_adjusted, xticks_adjusted = {}, {}
    vals_dict={}
    for HNL_mass in bins_dict:
        if isinstance(lower,(int, float)): lower_val = lower
        else: lower_val = bins_dict[HNL_mass][0]
        xlims_adjusted[HNL_mass] = [lower_val,bins_dict[HNL_mass][-1]]
        ticks = np.arange(bins_dict[HNL_mass][0], bins_dict[HNL_mass][-1], spacing)
        if ticks[-1] != bins_dict[HNL_mass][-2]: ticks = np.append(ticks, bins_dict[HNL_mass][-1]-1)
        ticks_strings = []
        vals = []
        for val in ticks:
            ticks_strings.append(str(int(val)))
            vals.append(val)
        ticks_strings[-1] = str(ticks_strings[-1])+"+"
        xticks_adjusted[HNL_mass] = ticks_strings
        vals_dict[HNL_mass] = vals
        
    return xlims_adjusted, xticks_adjusted, vals_dict

xlims_dict, xticks_dict, vals_dict = make_xlims_dict(bins_overflow, 2)

In [ ]:
mass = 150
plt.hist(Functions.logit(final_merged[f'BDT_output_{mass}MeV']),bins=bins_dict[mass])

## End of code